In [1]:
import pandas as pd
import os
import missingness_generation
#from src.cleaning import datacleaning
#from src import missingness_generation

In [2]:
# cleaned = datacleaning.cleaning(os.path.join(os.path.dirname(
#     os.path.realpath('run.py')) + '/data/test.csv')).iloc[:,1:]

In [3]:
data = pd.read_csv(os.path.join(os.path.dirname(
    os.path.realpath('run.py')) + '/data/allegations.csv')).iloc[:,1:]

In [4]:
data = data[['complaint_id', 'complainant_ethnicity', 'complainant_gender', 'complainant_age_incident',
       'allegation', 'contact_reason', 'Substantiated']]

In [5]:
data = data.rename(columns = {'Substantiated': 'substantiated'})

In [6]:
data.isna().sum()

complaint_id                   0
complainant_ethnicity       5505
complainant_gender          4195
complainant_age_incident    4829
allegation                     1
contact_reason               199
substantiated                  0
dtype: int64

In [7]:
data.head()

,complaint_id,complainant_ethnicity,complainant_gender,complainant_age_incident,allegation,contact_reason,substantiated
0,42835,Black,Female,38.0,Failure to provide RTKA card,Report-domestic dispute,True
1,24601,Black,Male,26.0,Action,Moving violation,True
2,24601,Black,Male,26.0,Race,Moving violation,True
3,26146,Black,Male,45.0,Question,PD suspected C/V of violation/crime - street,True
4,40253,NaN,NaN,16.0,Physical force,Report-dispute,True


We want the attribute with missingness to have around the same proportion of missingness for each type. This is because we don't want the amount of missingness to be a confounding factor in our results.

In [8]:
mcar = data.copy()
mcar = missingness_generation.mcar(mcar, 'substantiated')

/Users/alecpanattoni/Documents/MissingnessFairnessAnalysis/missingness_generation.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[column].iloc[i] = np.nan


In [9]:
mcar['substantiated'].isna().sum() / mcar.shape[0]

0.19725403201630792

In [29]:
mcar = mcar.dropna(subset = 'substantiated')

In [23]:
mar = data.copy()
mar = missingness_generation.mar(mar, 'substantiated', 'complainant_ethnicity', 0.3)

In [24]:
mar['substantiated'].isna().sum() / mar.shape[0]

0.18049643263984652

In [30]:
mar = mar.dropna(subset = 'substantiated')

In [21]:
nmar = data.copy()
nmar = missingness_generation.nmar(nmar, 'substantiated', 0.3)

In [22]:
nmar['substantiated'].isna().sum() / nmar.shape[0]

0.2046285748546076

In [31]:
nmar = nmar.dropna(subset = 'substantiated')

Now we will "handle" the missingness by dropping missing values.